In [1]:
import pandas as pd

# As operações de merge, join e concatenation
- Nesse notebook vamos explorar mais uma das operações mais importantes do Pandas
- O foco é juntar dois dataframes dado uma chave de conectora entre eles
- Aqui vamos carregar diferentes dataframes:

In [2]:
week_1 = pd.read_csv("../data/Restaurant - Week 1 Sales.csv")
week_1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [3]:
week_2 = pd.read_csv("../data/Restaurant - Week 2 Sales.csv")
week_2.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/Restaurant - Week 2 Sales.csv'

In [4]:
customers = pd.read_csv("../data/Restaurant - Customers.csv")
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [5]:
foods = pd.read_csv("../data/Restaurant - Foods.csv")
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


## Concatenação
- Vamos começar com a operação de concatenação. Como o nome sugere, ela vai concatenar um dataframe no final do outro
- Podemos obter esse resultado de duas formas: usando `concat()` ou `append()`

Vamos concatenar `week_1` com `week_2`:

In [6]:
pd.concat([week_1, week_2])

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


- Duas observações:
    - A ordem importa no sentido que um é concatenado no outro de acordo com a ordem dos parâmetros
    - Perceba que os indices estão repetindo, para resetar eles, podemos fazer:

In [7]:
pd.concat([week_1, week_2], ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


- A outra maneira de obter um resultado igual é usando o `append()`: 

In [8]:
week_1.append(week_2, ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


- Podemos obter um resultado usando multiindex:

In [9]:
pd.concat([week_1, week_2], keys=["week 1", "week 2"])

Customer ID  Food ID
week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

## Joins
- Não é intenção debater as definições basicas de um Join, mas podemos resumir com a seguinte figura:
        - Obs: o JOIN também pode ser chamado de INNER JOIN
<img src="../assets/sql-joins.png" alt="Joins" style="width: 40%;"/>


- A ideia é exatamente a mesma que a gente obtém no SQL
    - Vamos ter a tabela da direita e da esquerda
    - Precisamos falar qual JOIN queremos executar (INNER, LEFT, RIGHT...)
    - Precisamos uma chave para conectar as tabelas
- O método que executa essas operações no pandas é o `merge()`
    
### INNER JOIN
- Similar a uma operação `AND`. Vai fazer a interseção das tabelas de acordo com a chave passada
1. Vamos fazer um INNER JOIN entre os dataframes `week_1` e `week_2`:

In [10]:
week_1.merge(week_2, how="inner", on="Customer ID")

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


- Perceba que a interseção entre os dados foi feita baseado na coluna `Customer ID` como chave de conexão
- Observe que os indices `1` e `2` repetem o `Customer ID`. Isso acontece porque no DF `week_1`, esse cliente tem duas entradas. Então ele faz o produto cartesiano, ou seja, repete 2x o mesmo ID
- Observe também que foi acrescentado `_x` e `_y` nas colunas agregadas. Podemos controlar isso:

In [11]:
week_1.merge(week_2, how="inner", on="Customer ID", suffixes=[" W1", " W2"]).head()

,Customer ID,Food ID W1,Food ID W2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9


2. Agora imagine que desejamos obter um merge de todos os clientes que comeram a mesma comida na semana 1 e 2:

In [12]:
week_1.merge(week_2, how="inner", on=["Customer ID", "Food ID"])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


- Nesse caso nao precisamos de 2 colunas `Food` porque elas teriam o mesmo valor. O Pandas é esperto o suficiente pra eliminar uma delas
- Você pode verificar se os IDs possuem o mesmo pedido
- **Nota:** podemos usar o método merge direto do pandas -> `pd.merge()`
    - Nesse caso, a gente passa ambos os DFs como parametro. A sintaxe é a mesma. É apenas uma alternativa
    - Só precisamos respeitar a posição das tabelas (esquerda e direita)

In [38]:
pd.merge(week_1, week_2, how="inner", on=["Customer ID", "Food ID"]).head()

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4


## OUTER JOIN
- Essa operação vai juntar todas as informações dos DFs de acordo com a chave. É uma operação `OR`
- A sintaxe é quase a mesma do INNER

2. Vamos obter todos os pedidos de todos os clientes nas duas semanas (FULL OUTER INCLUSIVE)

In [13]:
week_1.merge(week_2, how="outer", on="Customer ID", suffixes=[" W1", " W2"])

,Customer ID,Food ID W1,Food ID W2
0,537,9.0,5.0
1,97,4.0,NaN
2,658,1.0,NaN
3,202,2.0,NaN
4,155,9.0,3.0
...,...,...,...
449,855,NaN,4.0
450,559,NaN,10.0
451,276,NaN,4.0
452,556,NaN,10.0


- Observe que existe index em que o cliente tem pedido em uma semana mas nao na outra
- Isso acontece porque é uma operação `OR`
- Podemos usar o parametro `indicator` para indicar de onde o valor está vindo

In [14]:
merged = week_1.merge(week_2, how="outer", on="Customer ID", suffixes=[" W1", " W2"], indicator=True)
merged.head()

,Customer ID,Food ID W1,Food ID W2,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both


In [15]:
merged["_merge"].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

2. Vamos fazer agora o FULL OUTER EXCUSIVE, ou seja, remover os valores que estão presentes nos dois ao mesmo tempo
- Neste caso, não temos um comando especifico, mas podemos obter da seguinte forma:

In [20]:
outer_exc_mask = merged["_merge"].isin(["left_only", "right_only"])
merged[outer_exc_mask].head()

,Customer ID,Food ID W1,Food ID W2,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only


### LEFT e RIGHT JOINS
- Como podemos ver na figura, a ideia dos dois joins é praticamente a mesma, só muda a posição dos conjuntos
    - Portanto, vamos fazer apenas para o LEFT porque é a mesma coisa
- Para obter é similar aos anteriores

3. Vamos fazer mesclar os dados de `week_1` com `foods` fazendo um LEFT JOIN
    - Ou seja, os dados em `foods` são relevantes, se existirem em `week_1`
    - Se os IDs não existirem em `foods` vai ser retornad `NaN`
    - Valores exclusivos de `foods` são irrelevantes
    - Em resumo, vai pegar todos os IDs em `week_1` e ve se acha em `foods`. Se achar, traz os dados. Se nao, coloca `NaN`

In [21]:
week_1.merge(foods, how="left", on="Food ID")

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
245,413,9,Donut,0.99
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99


- Pro tip: podemos ordernar o resultado de acordo com a chave informada (no caso `Food ID`):

In [22]:
week_1.merge(foods, how="left", on="Food ID", sort=True)

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


## Os parametros `left_on` e `right_on`
- O método `merge()` possui esses dois parametros que por padrão é `None`
- Até o momento, os DFs que fizemos merge compartilham o mesmo nome para a coluna que usamos como chave
- Esses parametros é justamente pra controlar isso quando as colunas contém a informação chave, mas possuem nomes diferentes

4. Vamos aplicar um LEFT JOIN entre os DFs `week_2` e `customers`
    - Nesse exemplo, o ID do cliente se chama `Customer ID` em `week_2` e apenas `ID` em `customers`

In [25]:
week_2.merge(customers, how="left", left_on='Customer ID', right_on='ID')

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,688,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,813,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,495,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,189,Roger,Gordon,Male,Skilith,Operator
4,267,3,267,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...,...
245,783,10,783,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,556,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,547,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,252,Douglas,Powell,Male,Jetwire,Geologist IV


- Observe que `left_on` está linkado com `week_2` e `right_on` com `costumers`. Isso é por conta da posição das chamadas
- Observe também que ele cria uma coluna para as duas chaves, que devem possuir o mesmo valor

### Executando o merge com indices e colunas
- Até o momento aplicamos o merge apenas com colunas. Porém, também é possível usar apenas indices ou mesclar os dois
- Isso é controlado com os parâmetros `right_index` e `left_index` quer por padrão são falsos, ou seja, não será usado indice pra eles

5. Vamos executar o merge anterior, mas usando indice para o DF `custormer`
    - Pra isso, vamos alterar o dataframe

In [31]:
customers.set_index("ID", inplace=True)
customers.head()

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
3,Roger,Black,Male,Tagfeed,Account Executive
4,Steven,Evans,Male,Fatz,Registered Nurse
5,Judy,Morrison,Female,Demivee,Legal Assistant


- Agora vamos fazer a mesma operação que foi feita em 4

In [32]:
week_2.merge(customers, how="left", left_on='Customer ID', right_on='ID', right_index=True)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,Roger,Gordon,Male,Skilith,Operator
4,267,3,Matthew,Wood,Male,Agimba,Product Engineer
...,...,...,...,...,...,...,...
245,783,10,Phyllis,Meyer,Female,Voolia,Information Systems Manager
246,556,10,Samuel,Bailey,Male,Oyoloo,Nurse
247,547,9,Tina,Watkins,Female,Thoughtstorm,Accountant II
248,252,9,Douglas,Powell,Male,Jetwire,Geologist IV


- Observe que falamos para o pandas olhar a chave `ID` na tabela `customers` através dos indices. Isso foi feito fazendo `right_index=True`

### O método `join()`
- O pandas também possui um método join, e isso pode ser confuso
- Na verdade, ele é como um concatenador de colunas baseado em indices
- É possível obter qualquer mesclagem com `merge()`, enquanto o `join()` serve apenas para casos específicos na qual vamos escrever menos codigo
- Para entender melhor vamos a um exemplo. Vamos carregar um dado novo que informa o nível de satisfação de cada cliente para a semana 1

In [35]:
satisfaction = pd.read_csv("../data/Restaurant - Week 1 Satisfaction.csv")
satisfaction.head()

,Satisfaction Rating
0,2
1,7
2,3
3,7
4,10


- Os indices desse dataframe estão pareados com os indices do `week_1`, ou seja, o indice 4 é o mesmo nas duas tabelas e assim por diante
- Podemos fazer um merge nessas tabelas da seguinte forma:

In [36]:
week_1.merge(satisfaction, how="left", left_index=True, right_index=True).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


- Nesse caso, obtemos o nivel de satisfação para cada cliente usando apenas os indices das duas tabelas, que novamente, são pareados
- É aqui que entra o `join()`: ele faz essa exata operação sem precisar passar nenhum parametro
- O importante aqui é que os indices sejam correspondentes

In [37]:
week_1.join(satisfaction).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
